## Tangency

We often discuss the efficient frontier for risky assets, but we can also consider the efficient frontier for the set of $N+1$ assets comprised of $N$ risky assets and the risk-free asset.  If it exists, the tangency portfolio is the efficient portfolio comprised solely of risky assets. 

Let $w_1,w_2,\dots,w_N$ denote portfolio weights in the risky assets.  Since the portfolio is fully invested, the allocation to the risk-free asset is $1-\sum_{i=1}^N w_i$.  The portfolio variance of the $N+1$ asset portfolio is $w'Vw$ where $w'=[w_1\, w_2\, \dots\,w_N]$.
The $N+1$ asset frontier is comprised of portfolios that minimize portfolio variance for a given target expected return $\mu_{\text{target}}$:

$$ \underset{w_1,w_2,\dots,w_N}{\text{min}} w'Vw$$
subject to constraints: $w' (\mu - r_f \mathbf{1}_N)=\mu_{\text{target}} - r_f\,.$
Note that the problem chooses only the risky asset weights because we have imposed the fully-invested constraint.  The first-order conditions of this constrained optimization problem are:

$$V w = \delta (\mu - r_f \mathbf{1}_N)$$
where $\delta$ is the Lagrange multiplier for the constraint.  Thus, the frontier portfolio $w$ solves the following system of equations:

\begin{equation*}
    \begin{bmatrix}
        \text{var}[r_1]     & \text{cov}[r_1,r_2] & \dots  & \text{cov}[r_1,r_N] \\
        \text{cov}[r_2,r_1] & \text{var}[r_2]     & \dots  & \text{cov}[r_2,r_N] \\
        \vdots              & \vdots              & \ddots &  \vdots \\
        \text{cov}[r_N,r_1] & \text{cov}[r_N,r_2] & \dots  & \text{var}[r_N] \\
    \end{bmatrix}
    \begin{bmatrix}
     w_1 \\ w_2 \\ \vdots \\ w_N
    \end{bmatrix}
    = \delta 
    \begin{bmatrix}
     \mu_1 - r_f \\ \mu_2 - r_f \\ \vdots \\ \mu_N - r_f
    \end{bmatrix}
\end{equation*}

We can rewrite this as:

\begin{align*}
    \sum_{i=1}^N \text{cov}[r_1,r_i] w_i &= \delta (\mu_1 - r_f) \\
    \sum_{i=1}^N \text{cov}[r_2,r_i] w_i &= \delta (\mu_2 - r_f) \\
    & \vdots \\
    \sum_{i=1}^N \text{cov}[r_N,r_i] w_i &= \delta (\mu_N - r_f) 
\end{align*}

We can view the terms on the left-hand side of the equations above as the contributions of each asset to overall portfolio risk.  The equations thus say that the ratio of an asset's excess return to its contribution to overall portfolio risk is the same across all assets for the optimal combination of risky assets.  If this were not the case for some pair of assets, then it would be possible to create a more efficient portfolio by investing a little more in the asset with the higher ratio and a little less in the asset with the lower ratio.


In [2]:

import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
##### Inputs
# Risk-free rate
r = 0.02
# Expected returns
means = np.array([0.06, 0.065, 0.08])
# Standard deviations
sds = np.array([0.15, 0.165, 0.21])
# Correlations
corr12 = 0.75
corr13 = 0.75
corr23 = 0.75
# Covariance matrix
C  = np.identity(3)
C[0, 1] = C[1, 0] = corr12
C[0, 2] = C[2, 0] = corr13
C[1, 2] = C[2, 1] = corr23
cov = np.diag(sds) @ C @ np.diag(sds)

The tangency problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{max}} \frac{E[r_p] - r_f}{\text{sd}[r_p]} $$ 
subject to the constraints $\sum_i w_i=1$.


In [16]:
# Tangency: theoretical solution without short-sale constraint
w = np.linalg.solve(cov, means - r)
wgts_tangency = w / np.sum(w)
print('Tangency portfolio is: ', wgts_tangency)

Tangency portfolio is:  [0.27606178 0.33045651 0.39348172]


Let's compare the ratios of excess return to the risk contribution for each asset.

In [12]:
def ratio(means, cov, rf, w):
    n = len(means)
    for i in np.arange(n):
        ratio = (means[i] - rf)/(cov[i,:] @ w)
        print(ratio)
ratio(means,cov, r, wgts_tangency)

1.848301193755739
1.848301193755739
1.848301193755739


In [15]:
ratio(means,cov, r, wgts_tangency + np.array([0.025, 0, -0.025]))
# cov

1.8507063472524783
1.862501355606362
1.8779127216551381
